In [1]:
import minsearch

In [2]:
import json

In [3]:
with open("../../documents.json", "rt") as f_in:
    docs_raw = json.load(f_in)

In [4]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [7]:
q = 'the course has already started, can I still enroll?'

In [8]:
index.fit(documents)

In [9]:
from openai import OpenAI

In [10]:
client = OpenAI()

In [11]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

In [12]:
response.choices[0].message.content

'Whether you can still enroll in a course that has already started depends on several factors, including the policies of the institution offering the course, the type of course, and how far along the course is. Here are some steps you can take:\n\n1. **Contact the Instructor or Institution**: Reach out to the course instructor or the admissions office to inquire about the possibility of late enrollment. Some courses may allow late enrollment, especially in the early weeks.\n\n2. **Check Online Platforms**: If the course is offered through an online platform, check their website or course page for information on late enrollment policies.\n\n3. **Understand the Impact**: If late enrollment is allowed, make sure you understand how it might impact your ability to catch up on missed content and assignments.\n\n4. **Consider Auditing**: If enrolling for credit is not possible, ask if you can audit the class. This might allow you to attend and participate without receiving credit or having as

In [13]:
def search(query): 
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [14]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION. 

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""

    for doc in search_results: 
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [15]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [16]:
query = 'how do i run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [17]:
rag(query)

'To run Kafka using Java, navigate to the project directory and execute the following command in the terminal:\n\n```shell\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```'

In [18]:
rag('the course has already started, can i still enroll?')

"Yes, you can still enroll in the course even after it has started. You are eligible to submit the homework assignments. However, please be aware that there are deadlines for submitting the final projects, so it's important not to delay too long."